In [1]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import snscrape.modules.twitter as sntwitter
import matplotlib.pyplot as plt
import seaborn as sns
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
os.chdir('/Users/alisdghnia/Desktop/School - Graduate/Statistical Programming for Business Analytics - Spring 2023/')

In [3]:
df = pd.read_csv('chatGPT.csv')
df.head(6)

,Date,Tweet,Url,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,https://twitter.com/heyandriadaily/status/1619...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
1,2023-01-29 15:07:59+00:00,I've been waiting for ChatGPT to open back up...,https://twitter.com/_doc_oc/status/16197139759...,_doc_oc,2022-02-08 01:15:45+00:00,False,24,144,0,0,Texas Medical Center,☢️ B.S. Nuclear Engineering \n🩺⚙️ Texas A&M E...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,https://twitter.com/nikoonya/status/1619713889...,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,https://twitter.com/CSOonline/status/161971369...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
4,2023-01-29 15:06:18+00:00,AI trainers = underpaid human annotators?!\n#C...,https://twitter.com/OmaymaS_/status/1619713552...,OmaymaS_,2009-12-13 15:58:13+00:00,False,1199,786,0,0,NaN,#datascience #machinelearning #rstats #python...
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,https://twitter.com/KevinPetschow/status/16197...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...


In [4]:
# create a new column for the tweetID
df['tweetID'] = np.nan

# tweetID is the numbers after the last slash in the url
for i in range(len(df)):
    df['tweetID'][i] = df['Url'][i].split('/')[-1]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/209259826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweetID'][i] = df['Url'][i].split('/')[-1]


In [5]:
data = []
for i, id in tqdm.tqdm(enumerate(df['tweetID']), total= len(df['tweetID'])):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data.append(new_data)

    if i > len(df['tweetID']):
        break

df2 = pd.DataFrame(data, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount', 
                                    'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

# add today's date to the dataframe
df2['date_scraped'] = pd.to_datetime('today').date()

# add a row to the dataframe in the beginning and assign null values since the first row is missing with the above code
df2.loc[-1] = np.nan
df2.index = df2.index + 1
df2 = df2.sort_index()

# save the dataframe as a csv file
df2.to_csv('chatGPT2.csv', index = False)

  0%|          | 0/179252 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/1642198707.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
Unavailable user in card on tweet 1619643336276709376
User 1350034417931075585 not found in user refs in card on tweet 1619643336276709376
Unavailable user in card on tweet 1619428852392153090
User 1616528732407169027 not found in user refs in card on tweet 1619428852392153090
Unavailable user in card on tweet 1619399147131666433
User 1530435498 not found in user refs in card on tweet 1619399147131666433
Unavailable user in card on tweet 1619316359711670273
User 1616528732407169027 not found in user refs in card on tweet 1619316359711670273
Unavailable user in card on tweet 1619061929296949248
User 3067393243 not found in user refs in card on tweet 1619061929296949248
Unavailable user in card on tweet 1618993380545

KeyboardInterrupt: 

In [7]:
len(data)

126074

In [8]:
test_data = pd.DataFrame(data, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                          'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [10]:
test_data.tail()

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
126069,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126070,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126071,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126072,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126073,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test_data.to_csv('test_data.csv')

In [18]:
data2 = []
for i, id in tqdm.tqdm(enumerate(df['tweetID'][126074:]), total= len(df['tweetID'])-126074):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data2.append(new_data)

    if i > len(df['tweetID'])-126074:
        break

  0%|          | 0/53178 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/3789065891.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
Unavailable user in card on tweet 1603024513784221697
Unavailable user in card on tweet 1603024513784221697
User 873939764876943363 not found in user refs in card on tweet 1603024513784221697
User 873939764876943363 not found in user refs in card on tweet 1603024513784221697
Unavailable user in card on tweet 1602695056523149318
User 1595487472297062400 not found in user refs in card on tweet 1602695056523149318
Unavailable user in card on tweet 1601921649820958721
User 3067393243 not found in user refs in card on tweet 1601921649820958721
Unavailable user in card on tweet 1601239597144764416
User 3067393243 not found in user refs in card on tweet 1601239597144764416


KeyError: 'result'

In [19]:
test_data2 = pd.DataFrame(data2, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                            'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [26]:
test_data2.to_csv('test_data2.csv')

In [27]:
len(data2) + len(data)

154084

In [28]:
data3 = []
for i, id in tqdm.tqdm(enumerate(df['tweetID'][154084:]), total= len(df['tweetID'])-154084):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data3.append(new_data)

    if i > len(df['tweetID'])-154084:
        break

  0%|          | 0/25168 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/1696863770.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
Unavailable user in card on tweet 1600042515670994944
User 633124104 not found in user refs in card on tweet 1600042515670994944
Unavailable user in card on tweet 1599661274962440193
User 1180544389778755584 not found in user refs in card on tweet 1599661274962440193
Unavailable user in card on tweet 1598654087314087936
User 933016351756742656 not found in user refs in card on tweet 1598654087314087936


In [29]:
test_data3 = pd.DataFrame(data3, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                            'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [32]:
len(df)

179252

In [34]:
nullCount = test_data.date.isnull().sum() + test_data2.date.isnull().sum() + test_data3.date.isnull().sum()

In [36]:
# number of remaining tweets
len(df) - nullCount

4235

In [66]:
scraper = sntwitter.TwitterTweetScraper(tweetId = 1619713543825403904)
scraper

In [67]:
scraper

In [83]:
for i in scraper.get_items():
    break
print(i.lang)

en
